In [1]:
import numpy as np
import math
from datascience import *
from scipy import stats
import pandas as pd
from pandas import *
import xlrd
from xlrd import open_workbook

### Load in the UN Gender Inequality Index (GII) data from 2016
Note: the table has been modified slightly from its original format for ease of use. The original table can be found at: http://hdr.undp.org/en/composite/GII

In [2]:
# cell for loading in dataset
# need to figure out if most elegant to import from excel -> pandas -> csv -> datascience package
# or if simply (ugly) dealing with float/string/nan issues are best dealt with by 
# looping through, making new columns and recreating table

The 'Value' and 'Rank' columns describe how each country did last year (2015). As you can see, the lower the value, the higher the rank. Thus, in 2016, Norway had the lowest value, although the raw value for 2016 is not shown in this table.

### How the GII is calculated:
![](gii_breakdown.png)

The Gender Inequality Index (GII) reflects gender-based disadvantage in three dimensions—reproductive health, empowerment and the labour market—for as many countries as data of reasonable quality allow. It shows the loss in potential human development due to inequality between female and male achievements in these dimensions. It ranges from 0, where women and men fare equally, to 1, where one gender fares as poorly as possible in all measured dimensions. (taken from UNDP technical notes)